# Nettoyage et analyse du dataset

In [132]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import plotly.graph_objects as go
import datetime
import matplotlib.pyplot as plt

##### Importation du csv dans un dataframe

In [133]:
df_ = pd.read_csv('./online_retail_II.csv')
df = df_.copy()

In [134]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Attribute Information:
- InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.
- Price: Unit price. Numeric. Product price per unit in sterling (Â£).
- CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal. The name of the country where a customer resides.

## Nettoyage des données

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


Différentes remarques sur ces informations :
- La colonne 'InvoiceDate' est de type objet au lieu d'être en date.  
- La plupart des colonnes ont 1067371 lignes à l'exception de 'Description' et 'Customer ID', il y a donc des informations manquantes. La colonne 'Description' n'est pas vraiment utile, mais celle des 'Customer ID' étant primordiale nous allons supprimer les lignes où l'ID manque.  
- La colonne 'Customer ID' est de type float64 alors qu'un type int64 est certainement suffisant.  
- Il n'y a pas de colonne pour décrire le prix total en fonction de la quantité d'articles achetée, ce qui pourrait être une information intéressante.  

### Suppression des lignes identiques

Voyons maintenant s'il y a des doublons, s'il y en a nous les supprimerons

In [136]:
df[df.duplicated()].shape

(34335, 8)

In [137]:
df = df.drop_duplicates(keep='first')
df[df.duplicated()].shape

(0, 8)

### Recherche et suppression des valeurs manquantes (Customer ID)

In [138]:
nan_count = df.isna().sum()
print(nan_count.sort_values)

<bound method Series.sort_values of Invoice             0
StockCode           0
Description      4275
Quantity            0
InvoiceDate         0
Price               0
Customer ID    235151
Country             0
dtype: int64>


In [139]:
df = df[df['Customer ID'].notna()]

### Conversion des données

In [140]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

In [141]:
df['Customer ID'] = df['Customer ID'].astype(np.int64)

### Nettoyage tourné "métier"

In [142]:
df.describe()

,Quantity,Price,Customer ID
count,797885.000000,797885.000000,797885.000000
mean,12.602980,3.702732,15313.062777
std,191.670371,71.392549,1696.466663
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13964.000000
50%,5.000000,1.950000,15228.000000
75%,12.000000,3.750000,16788.000000
max,80995.000000,38970.000000,18287.000000


On va essayer d'identifier dans le dataset les lignes qui ne correspondent pas à des achats

- Les prix négatifs correspondent à des annulations de commande, identifiées par un C dans le numéro de facture. On conserve ces lignes

- Certains articles sont au prix de 0€, il s'agit sans doute de cadeau, cela ne donne donc pas d'informations sur le comportement **d'achat** des clients. On va supprimer ces lignes

In [143]:
df= df[df['Price']> 0]

- On va recherche si certaines lignes correspondent à des tests ou des frais de ports

In [144]:
shipping_fees_data = df[df['Description'].str.contains('shipping|fees', case=False)]

print(shipping_fees_data)


Empty DataFrame
Columns: [Invoice, StockCode, Description, Quantity, InvoiceDate, Price, Customer ID, Country]
Index: []


On a remarqué que certains stocks code ne contiennent que des lettres (hors un stock code est censé être composé de 5 chiffres). On va essayer de comprendre à quoi correspondent ces stock codes.

In [145]:
letters_only_stockcodes = df[df['StockCode'].str.isalpha()]['StockCode'].unique()
print(letters_only_stockcodes)


['POST' 'D' 'M' 'PADS' 'ADJUST' 'DOT' 'CRUK']


- POST : Postage, ce code correspond aux frais de port, on décide de supprimer ces lignes

In [146]:
# On va afficher les descriptions uniques associées au Stock code POST
filtered_df = df[df['StockCode'] == 'POST']
unique_descriptions = filtered_df['Description'].unique()
for description in unique_descriptions:
    print(description)


POSTAGE


- M : Manual, correspondant à des lignes entrées manuellement, on converserve ces lignes

In [147]:
# On va afficher les descriptions uniques associées au Stock code M
filtered_df = df[df['StockCode'] == 'M']
unique_descriptions = filtered_df['Description'].unique()
for description in unique_descriptions:
    print(description)


Manual


- D : Discount : correspond à des remises, on supprime également ces lignes

In [148]:
# On va afficher les descriptions uniques associées au Stock code D
filtered_df = df[df['StockCode'] == 'D']
unique_descriptions = filtered_df['Description'].unique()
for description in unique_descriptions:
    print(description)

Discount


- PADS : Pads to match all cushions, cela semble correspondre à un produit, (pads = coussins), on conserve ces lignes

In [149]:
# On va afficher les descriptions uniques associées au Stock code PADS
filtered_df = df[df['StockCode'] == 'PADS']
unique_descriptions = filtered_df['Description'].unique()
for description in unique_descriptions:
    print(description)


PADS TO MATCH ALL CUSHIONS


- ADJUST : Adjustment by john on 26/01/2010 16	, ce code semble utiliser pour enregistrer les ajustements manuels ou les corrections apportées aux données, tels que les modifications de prix, les remises, les retours ou les corrections d'erreurs dans les transactions. On décide de supprimer ces lignes car elles ne donnent pas d'informations sur le comportement du client.

In [150]:
filtered_df = df[df['StockCode'] == 'ADJUST']
filtered_df.head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
70976,495733,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:21:00,68.34,14911,EIRE
70977,495735,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:22:00,201.56,12745,EIRE
70978,495734,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:22:00,205.82,14911,EIRE
70979,C495737,ADJUST,Adjustment by john on 26/01/2010 16,-1,2010-01-26 16:23:00,10.50,16154,United Kingdom
70980,495736,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:23:00,21.00,12606,Spain


- DOT : DOTCOM POSTAGE, cela doit correspondre à des frais de port à l'instar du code POST, on supprime ces lignes

- CRUK : CRUK Commission, on va supprimer ces lignes

In [154]:
# On va afficher les descriptions uniques associées au Stock code CRUK
unique_descriptions = filtered_df['Description'].unique()
for description in unique_descriptions:
    print(description)


CRUK Commission


### Feature Engineering

#### Feature : Total Price

Création d'une colonne 'Total' pour avoir le prix selon la quantité achetée

In [156]:
df['TotalPrice'] = df['Quantity'] * df['Price']
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0


#### Feature : Cancelled

Création d'une colonne "cancelled" (1 ou 0) pour pouvoir compter le nombre de retours effectués par chaque client au moment du groupby de manière plus pratique qu'avec le C dans le code facture.

In [157]:
df["cancelled"] = np.where(df["Quantity"] < 0, 1, 0)

In [158]:
# Trier le DataFrame par la colonne "Quantity" de manière décroissante
df_sorted = df.sort_values(by="Quantity", ascending=True)

# Afficher le DataFrame trié
df_sorted.head(100)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice,cancelled
1065883,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446,United Kingdom,-168469.60,1
587085,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346,United Kingdom,-77183.60,1
507225,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838,United Kingdom,-280.80,1
359669,C524235,21088,SET/6 FRUIT SALAD PAPER CUPS,-7128,2010-09-28 11:02:00,0.08,14277,France,-570.24,1
359670,C524235,21096,SET/6 FRUIT SALAD PAPER PLATES,-7008,2010-09-28 11:02:00,0.13,14277,France,-911.04,1
...,...,...,...,...,...,...,...,...,...,...
103623,C499282,21166,COOK WITH WINE METAL SIGN,-432,2010-02-25 18:55:00,1.69,16684,United Kingdom,-730.08,1
814595,C562221,22151,PLACE SETTING WHITE HEART,-432,2011-08-03 14:22:00,0.36,16525,United Kingdom,-155.52,1
845372,C564940,22564,ALPHABET STENCIL CRAFT,-432,2011-08-31 13:10:00,1.06,12931,United Kingdom,-457.92,1
252972,C513771,22333,RETRO SPORT PARTY BAG + STICKER SET,-432,2010-06-28 13:49:00,1.45,16754,United Kingdom,-626.40,1


Sauvegarde du dataframe (pour l'utiliser pour faire l'analyse RFM et clustering)

In [159]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 795569 entries, 0 to 1067369
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      795569 non-null  object        
 1   StockCode    795569 non-null  object        
 2   Description  795569 non-null  object        
 3   Quantity     795569 non-null  int64         
 4   InvoiceDate  795569 non-null  datetime64[ns]
 5   Price        795569 non-null  float64       
 6   Customer ID  795569 non-null  int64         
 7   Country      795569 non-null  object        
 8   TotalPrice   795569 non-null  float64       
 9   cancelled    795569 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 66.8+ MB


In [160]:
df.to_csv('clean_online_retail.csv', index=False)